# Making the Neural Network

This notebook contains explanations about the classes related to neural networks.

The Neural Network consists of two classes, network and layer located in the Network directory in the Network.py file. Each of these classes will be discussed in detail in their own section

### Helper functions

The Network file has two helper functions, ReLU and Sigmoid which take in a vector and apply the function to each value in the vector.

In [2]:
import numpy as np

def ReLU(x):
    return [xi if xi > 0 else 0 for xi in x]


def Sigmoid(x):
    return [0 if xi < 0 else 1 for xi in x]

### The Layer Class

The layer class represents each layer in the Neural Network. It consists of an activation function, a matrix of weights, and a vector of biases.

The layer class consists of two functions: activate and update.

Activate takes in a vector of input values and outputs the layer's function applied to the product of the inputs and the layer's weights plus the layer's bias vector.

Update takes in a mutation rate and a change value. It goes through each weight and bias, generating a random number for each. If this number is less than the mutation rate it adds a random number sampled from a uniform distribution that goes from (-change value, change value). This slightly modifies the network, possibly giving it better results.

In [3]:
class Layer:
    def __init__(self, bias, weights, function="ReLU"):
        self.function = function
        self.bias = bias
        self.weights = weights

    def activate(self, inputs):
        if self.function == "ReLU":
            return ReLU(np.dot(self.weights, inputs) + self.bias)
        if self.function == "Sigmoid":
            return Sigmoid(np.dot(self.weights, inputs) + self.bias)

    def update(self, mutation_rate, change_value):
        for i in range(len(self.weights)):
            for j in range(len(self.weights[i])):
                if np.random.random() < mutation_rate:
                    self.weights[i][j] += np.random.uniform(-change_value, change_value)
        for i in range(len(self.bias)):
            if np.random.random() < mutation_rate:
                self.bias[i] += np.random.uniform(-change_value, change_value)

### The Network Class

The network class consists of a list of layer objects. It can accept weights and bias vectors if you want to replicate a network, or it will randomly generate the values as shown in class. The last layer will have Sigmoid activation while all the other layers will have ReLU activation.

The network class has 3 functions: calc, update, and getWeights.

The calc function takes an input vector and feeds it through the network, returning the output.

The update function calls update on each layer of the network.

The getWeights function returns the weights and biases of the network as a tuple.

In [4]:
class Network:
    # Input_size is the size of the initial input data
    # Shape is an array of integers with each one containing the size of a layer for the network
    # kwargs will contain the bias and weights if you want to pre-initialize the network from a previous run
    def __init__(self, input_size, shape, **kwargs):

        bias = kwargs.get("bias", None)
        weights = kwargs.get("weights", None)
        self.layers = []
        previous_layer = input_size

        # Run through each layer that needs to be created
        for i in range(len(shape)):
            layer = shape[i]

            # Check if there are preset bias and weights, otherwise randomize them as shown in class
            if bias is not None and weights is not None:
                b = bias[i]
                w = weights[i]
            else:
                b = np.zeros(layer)
                w = np.random.default_rng().normal(loc=0, scale= 2 / (layer + previous_layer), size=(layer, previous_layer))

            # If it is the last layer use "Sigmoid", otherwise use the default of ReLU
            if i == len(shape) - 1:
                self.layers.append(Layer(bias=b, weights=w, function="Sigmoid"))
            else:
                self.layers.append(Layer(bias=b, weights=w))

            previous_layer = layer

    # Given the input, run through the network to get the output
    def calc(self, inputs):
        for layer in self.layers:
            inputs = layer.activate(inputs)
        return inputs

    # Update the weights and bias in the network by some random amount
    # Can accept a mutation_rate value which is the rate of change and a change_value which is the max amount of change
    def update(self, mutation_rate, change_value):
        for layer in self.layers:
            layer.update(mutation_rate, change_value)

    def getWeights(self):
        weights = []
        biases = []
        for layer in self.layers:
            weights.append(layer.weights)
            biases.append(layer.bias)
        return weights, biases

## Using the Network Class

The network class takes a value called input_size, which is the size of the input the network will receive and an array called shape which is a list of the size of each layer. As optional parameters you can pass in a tensor of weights and a matrix of biases to initialize the network with.

In [11]:
# Try playing around with various network and input sizes
network = Network(4, [8,7,6,1], bias=None, weights=None)
inputs = [1, 2, 3, 4]

output = network.calc(inputs)
print(output)


[0]
